<a href="https://colab.research.google.com/github/jikhans/python_algorthm_basic/blob/master/2019_04_24_VI_PYMC3%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VI : https://medium.com/@albertoarrigoni/scalable-bayesian-inference-in-python-a6690c7061a3

* Variational Inference in PyMC3

In [0]:
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np

def logistic(x, b, noise =None):
  L = x.T.dot(b)
  if noise is not None:
    L = L + noise # L may likelihood
  return 1/(1+np.exp(-L))


In [0]:
x1 = np.linspace(-10., 10, 10000)
x2 = np.linspace(0., 20, 10000)
bias = np.ones(len(x1))
X = np.vstack([x1,x2,bias])
B = [-10., 2., 1.] # sigmoid params for X + intercept


* noise mean

In [0]:
pnoisy = logistic(X,B, noise = np.random.normal(loc=0., scale=0., size = len(x1)))

 dichotomize pnoisy -- sample 0/1 with probability pnoisy

In [0]:
y =np.random.binomial(1., pnoisy)

*  What we are doing here is just creating two variables (x1, x2) whose linear combination is run through a sigmoid function.

* after that we sample from a Binomial distribution with parameter p defined by the sigmoid output. 

* The coefficients (betas) of the model are stored in the list ‘B’. 

* At this point we use Pymc3 to define a probabilistic model for logistic regression and try to obtain a posterior distribution for each of the parameters (betas) defined above.

In [0]:
with pm.Model() as model:
  # define prior
  intercept = pm.Normal('Intercept',0, sd=10)
  x1_coef = pm.Normal('x1', 0, sd=10)
  x2_coef = pm.Normal('x2', 0, sd=10)
  
  # Define likelihood
  
  likelhood = pm.Bernoulli('y',
                         pm.math.sigmoid(intercept + x1_coef*X[0] + x2_coef*X[1]), observed = y)
  trance = pm.sample(3000)
  #pm.traceplot(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [x2, x1, Intercept]
100%|██████████| 3500/3500 [09:11<00:00,  6.66it/s]
The number of effective samples is smaller than 25% for some parameters.


* The results are approximately what we expected: the maximum a posteriori (MAP) estimation coincides with the ‘beta’ parameters we used for data generation. 

In [0]:
# AVDI

from pymc3.variational.callbacks import CheckParametersConvergence

with model:
  fit = pm.fit(100_000, method ='advi', callbacks =[CheckParametersConvergence()])
  
draws = fit.sample(2_000) # This will automatically check parameters converge

import arviz as az

az.plot_forest([draws, trace])

Average Loss = 232.84:  54%|█████▍    | 54372/100000 [01:56<01:29, 509.05it/s]
Convergence achieved at 54400
Interrupted at 54,399 [54%]: Average Loss = 1,982.9


ModuleNotFoundError: ignored

In [0]:
az.plot_pair(trace, figsize =(5,5)) # covariance plots for the NUTS trace

In [0]:
az.plot_pair(draws, figsize =(5,5)) # covariance plots for the ADVI trace

* reference: https://stackoverflow.com/questions/52558826/why-is-pymc3-advi-worse-than-mcmc-in-this-logistic-regression-example

* ADVI is clearly underestimating the variance, but it is fairly close to the mean of each parameter.
* Let us try to visualize the covariance structure of the model to understand where this lack of precision may come from

* Clearly, ADVI does not capture (as expected) the interactions between variables because of the mean field approximation, 

* and so it underestimates the overall variance by far (be advised: this is a particularly tricky example chosen to highlight this kind of behavior).

* ADVI is a very convenient inferential procedure that let us characterize complex posterior distributions in a very short time (if compared to Gibbs/MCMC sampling). 

* The solution it finds is a distribution which approximate the posterior, although it may not converge to the real posterior: for most cases this may not be a problem,

* but we may need to pay extra-attention in cases where the covariance structure of the variables is crucial 